## Variant calling (with HaplotypeCaller)
save the following in a file named variant_calling.sh

In [ ]:
#!/bin/bash

# input bam file
f1=$1
# prefix of the output file
f2=$2
# directory where input files are saved
dir1=$3
# directory where output files will be saved
dir2=$4

cd ${dir2}

module load GATK
# on erisone the GATK jar is located at $EBROOTGATK
# (its full path is /apps/software/GATK/3.8-0-Java-1.8.0_161 )

refloc1=/data/humgen/burook/ref
refloc2=/pub/genome_references/gatk-bundle/1.5/hg19


# Variant calling

java \
   -jar /apps/software/GATK/3.8-0-Java-1.8.0_161/GenomeAnalysisTK.jar \
   -T HaplotypeCaller \
   -R ${refloc1}/genome.fa \
   -I ${f1} \
   --genotyping_mode DISCOVERY \
   -stand_call_conf 10 \
   --emitRefConfidence GVCF \
   -variant_index_type LINEAR -variant_index_parameter 128000 \
   --dbsnp ${refloc1}/dbsnp_135.hg19.vcf \
   --out ${f2}_recal3.vcf


run this script for individual bam files

In [ ]:
%%bash

# input data directory
dir1=/data/humgen/burook/sysbio_exome/gatk_BQSR/
# output data directory
dir2=/data/humgen/burook/sysbio_exome/gatk_indiv_vcf

#for f1 in ${dir1}/*marked_duplicates.bam 
for f1 in ${dir1}/*_recal3.bam 
do
    # f1 is the input sam file
    f2=$(basename $f1 _recal3.bam)
    
    bsub -m "hna001" -n 4 -M 8000 -o "tmp4.out" -e "tmp4.err" "sh variant_calling.sh $f1 $f2 $dir1 $dir2"

    sleep 1
done


In [ ]:
%%bash
# There seems to be duplicate sample names.
# Let's rename sample names with file names.

dir2=/data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf2
cd ${dir2}

for f1 in ${dir2}/*_recal3.vcf
do
    f2=$(basename $f1 .vcf)_ren.vcf
    oldName=$(grep 'CHR' ${f1} | cut -f10)
    newName=$(basename ${f1} _recal3.vcf)
    
    sed "0,/${oldName}/s//${newName}/" ${f1} > ${f2}
    
done

# Note: We do not need to do this for gatk4. 
# Instead we can pass a sample mapping file as an argument (--sample-name-map).

In [ ]:
%%bash
# Generate index file for vcf.
dir2=/data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf2
cd ${dir2}

printf '%s\n' '#!/bin/bash' '/PHShome/bm363/bin/gatk-4.1.7.0/gatk IndexFeatureFile --input $1' > index_vcf.sh


# now submit a job for each file
for f1 in ${dir2}/*_ren.vcf
do    
    if [ -f "${f1}.idx" ];
    then
    echo "it already exits"
    else
    bsub -m "hna001" -n 2 -M 8000 -o "tmpidx1.out" -e "tmpidx1.err" "sh index_vcf.sh ${f1}"
    fi
    sleep 1
done


## Joint Genotyping
First let's identify all gVCF files for sngle samples and list them in a single file.

In [51]:
%%bash
# a list of per-sample vcf files
cd /data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf2
find . -type f -name '*_ren.vcf' >> list_VCF1.list
sed -i -e 's/.\///g' list_VCF1.list

Save the following joint genotying scrintp in a file named joint_variant_calling.sh

In [ ]:
#!/bin/bash

# directory where input files are saved
dir1=$1

cd ${dir1}

module load GATK
# on erisone the GATK jar is located at $EBROOTGATK
# (its full path is /apps/software/GATK/3.8-0-Java-1.8.0_161 )


refloc1=/data/humgen/burook/ref
refloc2=/pub/genome_references/gatk-bundle/1.5/hg19


# Joint variant calling

java -jar /apps/software/GATK/3.8-0-Java-1.8.0_161/GenomeAnalysisTK.jar \
    -T GenotypeGVCFs \
    -R ${refloc1}/genome.fa \
    -V ${dir1}/list_VCF1.list \
    --dbsnp ${refloc1}/dbsnp_135.hg19.vcf \
    --out SysBio_exome_merged5.vcf \
    -nt 8


Now call this script from bash

In [ ]:
%%bash

# input data directory
dir1=/data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf2
# output data directory
dir2=/data/humgen/burook/sysbio_exome/gatk_merged_vcf
cd ${dir1}
bsub -m "hna001" -n 10 -M 200000 -o "tmp52.out" -e "tmp52.err" "sh /PHShome/bm363/WES_analysis/notebooks/joint_variant_calling_gatk3.sh  ${dir1}"

# on erisone generic node
bsub -q "big-multi" -n 8 -M 128000 -R "rusage[mem=128000]" -o "tmp52.out" -e "tmp52.err" "sh /PHShome/bm363/WES_analysis/notebooks/joint_variant_calling_gatk3.sh  ${dir1}"


# now let's move the output to the desired directory
mv ${dir1}/SysBio_exome_merged1.vcf ${dir2}/

In [49]:
%%bash
cd /data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf2

ls -lht | head


total 962G
-rw-r--r--. 1 bm363 humgen 3.5K May 17 04:15 tmp52.out
-rw-r--r--. 1 bm363 humgen 746K May 17 04:15 tmp52.err
-rw-r--r--. 1 bm363 humgen  12M May 17 04:14 SysBio_exome_merged11.vcf.idx
-rw-r--r--. 1 bm363 humgen  20G May 17 04:14 SysBio_exome_merged11.vcf
-rw-rw----. 1 bm363 humgen 4.2G May 13 18:48 SysBio_exome_merged5.vcf
-rw-r--r--. 1 bm363 humgen  11M May 12 17:02 212052_S5_recal3_ren.vcf.idx
-rw-r--r--. 1 bm363 humgen  19M May 12 17:01 212045_S4_recal3_ren.vcf.idx
-rw-r--r--. 1 bm363 humgen  11M May 12 16:58 212035_S6_recal3_ren.vcf.idx
-rw-r--r--. 1 bm363 humgen  15M May 12 16:57 212031_S1_recal3_ren.vcf.idx


In [45]:
%%bash
cd /data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf2
grep -v '#' SysBio_exome_merged11.vcf | cut -f1 | uniq -c

 374795 chr1
 340467 chr2
 293649 chr3
 240282 chr4
 226359 chr5
 251100 chr6
 246918 chr7
 210933 chr8
 180086 chr9
 218911 chr10
 211578 chr11
 209331 chr12
 121438 chr13
 128020 chr14
 149788 chr15
 152567 chr16
 166795 chr17
  99985 chr18
 153965 chr19
 101541 chr20
  61843 chr21
  78862 chr22
  75799 chrX
   1304 chrY


Multi-threading is not working with gatk3.8. Let's resubmit it with GATK4.

In [ ]:
# input data directory
dir1=/data/humgen/guffantilab/exome/SystemsBio/gatk_indiv_vcf2

bsub -m "hna001" -n 24 -M 128000 -o "tmpg4.out" -e "tmpg4.err" "sh /PHShome/bm363/WES_analysis/notebooks/joint_variant_calling_gatk3.sh  ${dir1}"


## Variant Recalibration

In [ ]:
#!/bin/bash

# directory where input files are saved
dir1=$1

cd ${dir1}

module load GATK
# on erisone the picard jar is located at $EBROOTGATK
# (its full path is /apps/software/GATK/3.8-0-Java-1.8.0_161 )

refloc1=/data/humgen/burook/ref
refloc2=/pub/genome_references/gatk-bundle/1.5/hg19


#### SNP's
# Variant recalibration of SNP's

java \
    -jar /apps/software/GATK/3.8-0-Java-1.8.0_161/GenomeAnalysisTK.jar \
    -T VariantRecalibrator \
    -R ${refloc1}/genome.fa \
    -input SysBio_exome_merged11.vcf \
    -resource:hapmap,known=false,training=true,truth=true,prior=15.0 ${refloc1}/hapmap_3.3.hg19.vcf \
    -resource:omni,known=false,training=true,truth=true,prior=12.0 ${refloc1}/1000G_omni2.5.hg19.vcf \
    -resource:1000G,known=false,training=true,truth=false,prior=10.0 ${refloc1}/Mills_and_1000G_gold_standard.indels.hg19.vcf \
    -resource:dbsnp,known=true,training=false,truth=false,prior=7 ${refloc1}/dbsnp_135.hg19.vcf \
    -an QD -an MQRankSum -an ReadPosRankSum -an FS -an MQ -an SOR -an DP \
    -mode SNP \
    --maxGaussians 6 \
    -tranche 100.0 -tranche 99.9 -tranche 99.0 -tranche 90.0 \
    -recalFile SysBio_snps_recal \
    -tranchesFile SysBio_snps_tranches \
    -rscriptFile SysBio_recal_snps_plots.R



# Apply Recalibration of SNP's

java \
    -jar /apps/software/GATK/3.8-0-Java-1.8.0_161/GenomeAnalysisTK.jar \
    -T ApplyRecalibration \
    -R ${refloc1}/genome.fa \
    -input SysBio_exome_merged11.vcf \
    -mode SNP \
    --ts_filter_level 99.0 \
    -recalFile SysBio_snps_recal \
    -tranchesFile SysBio_snps_tranches \
    --out SysBio_snps_recalibrated.vcf


#### INDELs
# Variant recalibration of INDELs

java \
    -jar /apps/software/GATK/3.8-0-Java-1.8.0_161/GenomeAnalysisTK.jar \
    -T VariantRecalibrator \
    -R ${refloc1}/genome.fa \
    -input SysBio_snps_recalibrated.vcf \
    -resource:1000G,known=false,training=true,truth=false,prior=12.0 ${refloc1}/Mills_and_1000G_gold_standard.indels.hg19.vcf \
    -resource:dbsnp,known=true,training=false,truth=false,prior=2.0 ${refloc1}/dbsnp_135.hg19.vcf \
    -an FS -an ReadPosRankSum -an MQRankSum -an QD -an SOR -an DP \
    -mode INDEL \
    -tranche 100.0 -tranche 99.9 -tranche 99.0 -tranche 90.0 \
    --maxGaussians 4 \
    -recalFile SysBio_indels_recal \
    -tranchesFile SysBio_indels_tranches \
    -rscriptFile SysBio_recal_indels_plots.R



# Apply Recalibration of SNP's

java \
    -jar /apps/software/GATK/3.8-0-Java-1.8.0_161/GenomeAnalysisTK.jar \
    -T ApplyRecalibration \
    -R ${refloc1}/genome.fa \
    -input SysBio_snps_recalibrated.vcf \
    -mode INDEL \
    --ts_filter_level 99.0 \
    -recalFile SysBio_indels_recal \
    -tranchesFile SysBio_indels_tranches \
    --out SysBio_indels_recalibrated.vcf


Now call this script

In [ ]:
%%bash

# input data directory
dir1=/data/humgen/guffantilab/exome/SystemsBio/gatk_merged_vcf

bsub -m "hna001" -n 24 -M 64000 -o "tmp7.out" -e "tmp7.err" "sh /PHShome/bm363/WES_analysis/notebooks/variant_recalibration.sh  ${dir1}"


In [74]:
%%bash
cd /data/humgen/guffantilab/exome/SystemsBio/gatk_merged_vcf

ls -lht 


# let's plot the results
plot-vcfstats SysBio_variant_stats.vchk -p stats_plots/
# The plots are not being generated due to an issue with Python.
# So let's generate it manually using the script generated.
cd stats_plots/
python plot.py



total 74G
-rw-r--r--. 1 bm363 humgen 3.2K May 17 12:33 tmp7.out
-rw-r--r--. 1 bm363 humgen  44K May 17 12:33 tmp7.err
-rw-r--r--. 1 bm363 humgen  12M May 17 12:33 SysBio_indels_recalibrated.vcf.idx
-rw-r--r--. 1 bm363 humgen  21G May 17 12:33 SysBio_indels_recalibrated.vcf
-rw-r--r--. 1 bm363 humgen 3.0M May 17 12:30 SysBio.indels.recal.idx
-rw-r--r--. 1 bm363 humgen  46M May 17 12:30 SysBio.indels.recal
-rw-r--r--. 1 bm363 humgen 2.8M May 17 12:30 SysBio_recal_indels_plots.R
-rw-r--r--. 1 bm363 humgen  597 May 17 12:29 SysBio.indels.tranches
-rw-r--r--. 1 bm363 humgen  12M May 17 12:25 SysBio_snps_recalibrated.vcf.idx
-rw-r--r--. 1 bm363 humgen  21G May 17 12:25 SysBio_snps_recalibrated.vcf
-rw-r--r--. 1 bm363 humgen  12M May 17 12:21 SysBio.snps.recal.idx
-rw-r--r--. 1 bm363 humgen 272M May 17 12:21 SysBio.snps.recal
-rw-r--r--. 1 bm363 humgen 2.8M May 17 12:21 SysBio_recal_snps_plots.R
-rw-r--r--. 1 bm363 humgen  584 May 17 12:21 SysBio.snps.tranches
-rw-r--r--. 1 bm363 humgen  12M 

## Liftover to b37

We need to use b37 reference to work with Hail. Since our mapping was done with hg19, let's liftover it to b37. Note that hg19 and b37 are similar references with little difference in annotation. The main difference being annotation of contigs (e.g., ch1 vs 1).

Download the chain file from the following 
ftp://gsapubftp-anonymous@ftp.broadinstitute.org/Liftover_Chain_Files. (need login. saved at /data/humgen/burook/ref/hg19tob37.chain)


In [ ]:
%%bash

module load picard

java -jar /apps/lib-osver/picard/2.22.0/build/libs/picard.jar LiftoverVcf\
    I=/data/humgen/guffantilab/exome/SystemsBio/gatk_merged_vcf/SysBio_indels_recalibrated.vcf \
    O=/data/humgen/guffantilab/exome/SystemsBio/gatk_merged_vcf/SysBio_indels_recalibrated_b37.vcf \
    CHAIN=/data/humgen/burook/ref/hg19tob37.chain \
    REJECT=rejected_variants.vcf \
    R=/data/humgen/burook/ref/human_g1k_v37.fasta
